Front Matter
========

This is a quick notebook to explore the outputs of the Keras model.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
print(tf.__version__)

import pandas as pd
import seaborn as sns

%load_ext tensorboard
%matplotlib inline


Make an example dataset
=========

In [ ]:
nPoints = 1000
x_sig = 1 * np.random.randn(nPoints) + 1
y_sig = 1 * np.random.randn(nPoints) + 1
x_bkg = 1 * np.random.randn(nPoints*10) - 1
y_bkg = 1 * np.random.randn(nPoints*10) - 1
label = np.append(np.ones(len(x_sig)),np.zeros(len(x_bkg))).astype(int)

In [ ]:
df = pd.DataFrame(list(zip(np.append(x_sig,x_bkg),
                           np.append(y_sig,y_bkg),
                           label)),columns=['x','y','label'])

In [ ]:
sns.pairplot(df,vars=['x', 'y'],hue='label');

Plot correlations of variables
----------

In [ ]:
tmp = sns.heatmap(df[df['label'] == 1][['x','y']].corr(),annot=True,vmin=-1,vmax=1)
b, t = plt.ylim() # discover the values for bottom and top
plt.ylim(b + 0.5, t - 0.5) # update the ylim(bottom, top) values
plt.show()

Make the Keras Model, compile and fit
-------------

In [ ]:
model = tf.keras.models.Sequential([
#    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(5, name='first_dense', input_shape=(2,), activation='relu'),
    tf.keras.layers.Dense(5, name='middle_dense', activation='relu'),
#    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, name='last_dense', activation='sigmoid') # sigmoid for binary; softmax for multi?
    ])

##### TODO:
#probability_model = tf.keras.Sequential([
#  model,
#  tf.keras.layers.Softmax()
#])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(
    df[['x','y']].values,
    df['label'].values,
    batch_size=64,
    epochs=20,
    verbose=2)

Visualize the Model Structure
----------

In [ ]:
#import pydot as pyd
#import graphviz
#tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True,
#    rankdir='TB', expand_nested=True, dpi=96)
#model.summary()

In [ ]:
# version 1
# from IPython.display import SVG
# SVG(tf.keras.utils.model_to_dot(model,expand_nested=True).create(prog='dot', format='svg'))
# version 2
#tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True,expand_nested=True)

Evaluate the performance
===========

In [ ]:
print(model.metrics_names)
model.evaluate(df[['x','y']].values,df['label'].values,verbose=2)

In [ ]:
# This sets things as 1 or 0
# based on whether or not is is above or below 0.5
df['binary_prediction'] = model.predict_classes(df[['x','y']].values)
# This is the continuous variable
df['discriminant'] = model.predict(df[['x','y']].values)

Confusion Matrix
-------

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(df['binary_prediction'].values,df['label'].values)

Plotting the raw discriminants
--------------

In [ ]:
plt.hist(df[df['label'] ==0]['discriminant'],bins=20,label='bkg',density=True,histtype='step',lw=2)
plt.hist(df[df['label'] ==1]['discriminant'],bins=20,label='sig',density=True,histtype='step',lw=2)
plt.legend()
plt.yscale('log')

ROC Curve
----------

In [ ]:
from sklearn.metrics import roc_curve
fpr_keras, tpr_keras, thresholds_keras = roc_curve(df['label'].values, df['discriminant'].values)
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.ylabel('signal efficiency (true positive rate)')
plt.xlabel('bkg efficiency (false positive rate)')

2-D Gradients
----------

In [ ]:
#Since probability is a continuous value from 0 to 1, we are getting many contours.
#If your visualization is restricted to 2 classes (output is 2D softmax vector) you can use this simple code

def plotModelOut(x,y,model):
    '''
    x,y: 2D MeshGrid input
    model: Keras Model API Object
    '''
    grid = np.stack((x,y))
    grid = grid.T.reshape(-1,2)
    outs = model.predict(grid)
    y1 = outs.T[0].reshape(x.shape[0],x.shape[0])
    plt.contourf(x,y,y1,cmap='binary')
    plt.colorbar()

    #plt.show()

xx, yy = np.meshgrid(np.linspace(-4,4,100), np.linspace(-4,4,100))
plotModelOut(xx,yy,model)

plt.scatter(df['x'][df['label'] == 0],df['y'][df['label'] == 0])
plt.scatter(df['x'][df['label'] == 1],df['y'][df['label'] == 1])

plt.xlim([-4,4])
plt.ylim([-4,4])
plt.show()